<a href="https://colab.research.google.com/github/abd-RAHEEM/Devign_implementation/blob/main/DevignDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers datasets scikit-learn


In [1]:
DATA_PATH='/content/drive/MyDrive/devign/'

In [2]:
import json

# load full dataset
with open(DATA_PATH + "Devign.json", "r") as f:
    data = json.load(f)

print("Total samples:", len(data))
print("Example:", data[0])


Total samples: 27318
Example: {'project': 'FFmpeg', 'commit_id': '973b1a6b9070e2bf17d17568cbaf4043ce931f51', 'target': 0, 'func': 'static av_cold int vdadec_init(AVCodecContext *avctx)\n\n{\n\n    VDADecoderContext *ctx = avctx->priv_data;\n\n    struct vda_context *vda_ctx = &ctx->vda_ctx;\n\n    OSStatus status;\n\n    int ret;\n\n\n\n    ctx->h264_initialized = 0;\n\n\n\n    /* init pix_fmts of codec */\n\n    if (!ff_h264_vda_decoder.pix_fmts) {\n\n        if (kCFCoreFoundationVersionNumber < kCFCoreFoundationVersionNumber10_7)\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts_prior_10_7;\n\n        else\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts;\n\n    }\n\n\n\n    /* init vda */\n\n    memset(vda_ctx, 0, sizeof(struct vda_context));\n\n    vda_ctx->width = avctx->width;\n\n    vda_ctx->height = avctx->height;\n\n    vda_ctx->format = \'avc1\';\n\n    vda_ctx->use_sync_decoding = 1;\n\n    vda_ctx->use_ref_buffer = 1;\n\n    ctx->pix_fmt = avctx->get_format

In [3]:
def read_split(file_path):
    with open(file_path, "r") as f:
        return set(line.strip() for line in f.readlines())

train_ids = read_split(DATA_PATH + "train.txt")
valid_ids = read_split(DATA_PATH + "valid.txt")
test_ids  = read_split(DATA_PATH + "test.txt")

print("Train:", len(train_ids), "Valid:", len(valid_ids), "Test:", len(test_ids))


Train: 21854 Valid: 2732 Test: 2732


In [4]:
print(data[0].keys())
print(data[0])


dict_keys(['project', 'commit_id', 'target', 'func'])
{'project': 'FFmpeg', 'commit_id': '973b1a6b9070e2bf17d17568cbaf4043ce931f51', 'target': 0, 'func': 'static av_cold int vdadec_init(AVCodecContext *avctx)\n\n{\n\n    VDADecoderContext *ctx = avctx->priv_data;\n\n    struct vda_context *vda_ctx = &ctx->vda_ctx;\n\n    OSStatus status;\n\n    int ret;\n\n\n\n    ctx->h264_initialized = 0;\n\n\n\n    /* init pix_fmts of codec */\n\n    if (!ff_h264_vda_decoder.pix_fmts) {\n\n        if (kCFCoreFoundationVersionNumber < kCFCoreFoundationVersionNumber10_7)\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts_prior_10_7;\n\n        else\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts;\n\n    }\n\n\n\n    /* init vda */\n\n    memset(vda_ctx, 0, sizeof(struct vda_context));\n\n    vda_ctx->width = avctx->width;\n\n    vda_ctx->height = avctx->height;\n\n    vda_ctx->format = \'avc1\';\n\n    vda_ctx->use_sync_decoding = 1;\n\n    vda_ctx->use_ref_buffer = 1;\n\n    ctx->pix

In [5]:
def make_dataset(data, train_ids, valid_ids, test_ids):
    train_data, valid_data, test_data = [], [], []
    for item in data:
        # use commit_id instead of id
        example_id = str(item.get("id", item.get("commit_id", "")))
        obj = {"id": example_id, "func": item["func"], "label": int(item["target"])}

        if example_id in train_ids:
            train_data.append(obj)
        elif example_id in valid_ids:
            valid_data.append(obj)
        elif example_id in test_ids:
            test_data.append(obj)

    return DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(valid_data),
        "test": Dataset.from_list(test_data)
    })


In [6]:
print(len(data))         # how many samples
print(data[0].keys())    # what fields each sample has
print(data[0])           # print the first sample fully


27318
dict_keys(['project', 'commit_id', 'target', 'func'])
{'project': 'FFmpeg', 'commit_id': '973b1a6b9070e2bf17d17568cbaf4043ce931f51', 'target': 0, 'func': 'static av_cold int vdadec_init(AVCodecContext *avctx)\n\n{\n\n    VDADecoderContext *ctx = avctx->priv_data;\n\n    struct vda_context *vda_ctx = &ctx->vda_ctx;\n\n    OSStatus status;\n\n    int ret;\n\n\n\n    ctx->h264_initialized = 0;\n\n\n\n    /* init pix_fmts of codec */\n\n    if (!ff_h264_vda_decoder.pix_fmts) {\n\n        if (kCFCoreFoundationVersionNumber < kCFCoreFoundationVersionNumber10_7)\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts_prior_10_7;\n\n        else\n\n            ff_h264_vda_decoder.pix_fmts = vda_pixfmts;\n\n    }\n\n\n\n    /* init vda */\n\n    memset(vda_ctx, 0, sizeof(struct vda_context));\n\n    vda_ctx->width = avctx->width;\n\n    vda_ctx->height = avctx->height;\n\n    vda_ctx->format = \'avc1\';\n\n    vda_ctx->use_sync_decoding = 1;\n\n    vda_ctx->use_ref_buffer = 1;\n\n    ct

In [7]:
from datasets import Dataset, DatasetDict

def make_dataset(data, train_ids, valid_ids, test_ids):
    train_data, valid_data, test_data = [], [], []
    for item in data:
        example_id = str(item["commit_id"])  # unique ID
        obj = {
            "id": example_id,
            "func": item["func"],          # source code
            "label": int(item["target"])   # 0 or 1
        }

        if example_id in train_ids:
            train_data.append(obj)
        elif example_id in valid_ids:
            valid_data.append(obj)
        elif example_id in test_ids:
            test_data.append(obj)

    return DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(valid_data),
        "test": Dataset.from_list(test_data)
    })


In [9]:
from transformers import RobertaTokenizerFast

# Use pretrained CodeBERT tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("microsoft/codebert-base")

# Check tokenizer
print(tokenizer.tokenize("int main() { return 0; }"))


['int', 'Ġmain', '()', 'Ġ{', 'Ġreturn', 'Ġ0', ';', 'Ġ}']


In [10]:
from datasets import Dataset, DatasetDict
import json

# Load your Devign.json
with open("/content/drive/MyDrive/devign/Devign.json", "r") as f:
    data = json.load(f)

# Replace 'id' with 'commit_id' since that exists in the JSON
def make_dataset(data, train_ids, valid_ids, test_ids):
    train_data, valid_data, test_data = [], [], []
    for item in data:
        example_id = str(item["commit_id"])  # unique ID
        obj = {
            "id": example_id,
            "func": item["func"],          # source code
            "label": int(item["target"])   # 0 or 1
        }

        if example_id in train_ids:
            train_data.append(obj)
        elif example_id in valid_ids:
            valid_data.append(obj)
        elif example_id in test_ids:
            test_data.append(obj)

    return DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(valid_data),
        "test": Dataset.from_list(test_data)
    })

# Define your train/valid/test splits (just an example, you can adjust)
all_ids = [str(item["commit_id"]) for item in data]
train_ids = all_ids[:20000]
valid_ids = all_ids[20000:24000]
test_ids  = all_ids[24000:]

# Create the dataset
dataset = make_dataset(data, train_ids, valid_ids, test_ids)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 24665
    })
    validation: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 1502
    })
    test: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 1151
    })
})


In [11]:
def tokenize_function(examples):
    return tokenizer(
        examples["func"],         # source code field
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)


Map:   0%|          | 0/24665 [00:00<?, ? examples/s]

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 24665
    })
    validation: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1502
    })
    test: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1151
    })
})


In [12]:
from datasets import Dataset, DatasetDict
import json

# Load your Devign.json
with open("/content/drive/MyDrive/devign/Devign.json", "r") as f:
    data = json.load(f)

# Replace 'id' with 'commit_id' since that exists in the JSON
def make_dataset(data, train_ids, valid_ids, test_ids):
    train_data, valid_data, test_data = [], [], []
    for item in data:
        example_id = str(item["commit_id"])  # unique ID
        obj = {
            "id": example_id,
            "func": item["func"],          # source code
            "label": int(item["target"])   # 0 or 1
        }

        if example_id in train_ids:
            train_data.append(obj)
        elif example_id in valid_ids:
            valid_data.append(obj)
        elif example_id in test_ids:
            test_data.append(obj)

    return DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(valid_data),
        "test": Dataset.from_list(test_data)
    })

# Define your train/valid/test splits (just an example, you can adjust)
all_ids = [str(item["commit_id"]) for item in data]
train_ids = all_ids[:20000]
valid_ids = all_ids[20000:24000]
test_ids  = all_ids[24000:]

# Create the dataset
dataset = make_dataset(data, train_ids, valid_ids, test_ids)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 24665
    })
    validation: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 1502
    })
    test: Dataset({
        features: ['id', 'func', 'label'],
        num_rows: 1151
    })
})


In [13]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("microsoft/codebert-base")

def tokenize_function(examples):
    return tokenizer(
        examples["func"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)


Map:   0%|          | 0/24665 [00:00<?, ? examples/s]

Map:   0%|          | 0/1502 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 24665
    })
    validation: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1502
    })
    test: Dataset({
        features: ['id', 'func', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1151
    })
})


In [15]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=2   # 0 = safe, 1 = vulnerable
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
!pip install --upgrade transformers datasets


In [21]:
!pip uninstall -y transformers
!pip uninstall -y tokenizers


Found existing installation: transformers 4.56.2
Uninstalling transformers-4.56.2:
  Successfully uninstalled transformers-4.56.2
Found existing installation: tokenizers 0.22.0
Uninstalling tokenizers-0.22.0:
  Successfully uninstalled tokenizers-0.22.0


In [15]:
!pip install transformers==4.45.0
!pip install datasets --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# Metrics function
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[]  # <--- This disables wandb/other loggers
)


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.694800,0.718834,0.480692,0.007634
1000,0.670800,0.669430,0.553262,0.250279
1500,0.678200,0.637633,0.559254,0.267699
2000,0.662100,0.639084,0.562583,0.277228
2500,0.698600,0.709231,0.478695,0.000000
3000,0.689700,0.720309,0.478695,0.000000
3500,0.697000,0.695096,0.478695,0.000000
4000,0.693500,0.697106,0.478695,0.000000
4500,0.696100,0.699137,0.478695,0.000000
5000,0.691500,0.705903,0.478695,0.000000


TrainOutput(global_step=9252, training_loss=0.6877589388631713, metrics={'train_runtime': 4108.1947, 'train_samples_per_second': 18.012, 'train_steps_per_second': 2.252, 'total_flos': 9734451270681600.0, 'train_loss': 0.6877589388631713, 'epoch': 3.0})

In [21]:
metrics = trainer.evaluate(tokenized_datasets["test"])
print(metrics)


{'eval_loss': 0.6650668382644653, 'eval_accuracy': 0.4926151172893136, 'eval_f1': 0.22340425531914893, 'eval_runtime': 16.6284, 'eval_samples_per_second': 69.219, 'eval_steps_per_second': 8.66, 'epoch': 3.0}


In [25]:
import torch

def predict_code(code_snippet):
    # prepare inputs
    inputs = tokenizer(
        code_snippet,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    # move inputs to same device as model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # forward pass
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    pred_label = torch.argmax(probs, dim=-1).item()
    return pred_label, probs.detach().cpu().numpy()


In [26]:
sample_code = """
int main() {
    char buf[10];
    gets(buf); // unsafe, possible buffer overflow
    return 0;
}
"""

label, prob = predict_code(sample_code)
print("Predicted label:", label, "Probabilities:", prob)


Predicted label: 1 Probabilities: [[0.00817176 0.9918282 ]]


In [27]:
# create a folder for your model
save_dir = "./saved_devign_model"

# save model and tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('./saved_devign_model/tokenizer_config.json',
 './saved_devign_model/special_tokens_map.json',
 './saved_devign_model/vocab.json',
 './saved_devign_model/merges.txt',
 './saved_devign_model/added_tokens.json',
 './saved_devign_model/tokenizer.json')

In [32]:
new_code = """
void Test() {
    char buff[5];
    char buff2[3];
    printf("Enter some input: ");
    // Use fgets to limit input size and avoid overflow
    if (fgets(buff, sizeof(buff), stdin) != NULL) {
        // Remove newline if present
        buff[strcspn(buff, "\n")] = '\0';

        // Use strncpy to copy safely with size limit
        strncpy(buff2, buff, sizeof(buff2) - 1);
        buff2[sizeof(buff2) - 1] = '\0';  // Null-terminate explicitly

        printf("buff2 is %s\n", buff2);
    } else {
        printf("Input error\n");
    }
}

"""
label, prob = predict_code(new_code)
print("Predicted label:", label, "Probabilities:", prob)


Predicted label: 1 Probabilities: [[0.0081723 0.9918277]]
